[View in Colaboratory](https://colab.research.google.com/github/Giffy/CarCrashDetector/blob/master/1_Building_a_Dataset.ipynb)

# Index
<ol>
    <li><a href="#env_setup">Environment setup </a>
    <li><a href="#drive_setup">Connection to Google drive</a>
    <li><a href="#variables">Constants and variables</a>
    <li><a href="#video_frame">Transform videos to frames</a>

<a id="env_setup"> </a>
## 1. Environment setup and library import

In [0]:
!pip install --upgrade pip > /dev/null
!pip install scikit-image > /dev/null

In [0]:
import cv2
#import skimage
from skimage.color import rgb2gray
from skimage.transform import resize
import matplotlib.pyplot as plt
import math

<a id="drive_setup"> </a>
## 2. Link Goggle Drive with Colab

Run the code and follow the link to get an authentification key, copy it and paste in the box that will appear in jupyter notebook. After fist key the script will ask for a second authentification key, follow the process as above.

Source : https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d 

In [0]:
# Check is Link to Drive is OK
google = !if [ -d 'mydrive/' ]; then echo "1" ; else echo "0"; fi
if (google[0] is '0' ):
  !apt-get install -y -qq software-properties-common python-software-properties module-init-tools > /dev/null
  !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
  !apt-get update -qq 2>&1 > /dev/null
  !apt-get -y install -qq google-drive-ocamlfuse fuse > /dev/null
  from google.colab import auth
  auth.authenticate_user()
  from oauth2client.client import GoogleCredentials
  creds = GoogleCredentials.get_application_default()
  import getpass
  !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
  vcode = getpass.getpass()
  !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
  # Makes folder 'mydrive' in colab and mounts a link to 'Google drive'
  !mkdir -p mydrive
  !google-drive-ocamlfuse mydrive
!if [ -d 'mydrive/' ]; then echo "Connection to Google drive successful" ; else echo "Error to connect to Google drive"; fi

<a id="variables"> </a>
## 3. Constants and variables

In [0]:
INPUT_VIDEOS_PATH = 'mydrive/CarCrashDetection/Sources'              # Path to folder with videos 
OUTPUT_FRAMES_PATH = 'mydrive/CarCrashDetection/Sources/frames'      # Location of extracted images

frame_name = 'frame'                                                 # Frame name       
one_frame_each = 4                                                   # Extract one frame each 70



!if [ -d {OUTPUT_FRAMES_PATH} ]; then echo "Output to be stored in "{OUTPUT_FRAMES_PATH} ; else mkdir {OUTPUT_FRAMES_PATH} && echo "Output directory created"; fi

files = !ls {INPUT_VIDEOS_PATH}/*.mp4                                # Video file names in INPUT VIDEOS PATH
videofile = files[0]                                                     # Refactor of variable 'files' to convert the first video of list 

<a id="video_frame"> </a>
## 4. Transform videos to frames

In [0]:
count = 0
frame_name = frame_name + version
success = True

vidcap = cv2.VideoCapture(videofile)

while success:
    if (count%one_frame_each == 0):                                   # checks frame number and keeps one_frame_each          
        success,image = vidcap.read()                                 # reads next frame           
        image_gray = rgb2gray(image)                                  # grayscale image
        if image.shape[1]>700:                                        # if image width > 700, resize it
          tmp = resize(image_gray, (math.floor(640 / image_gray.shape[1] * image_gray.shape[0]), 640),mode='constant')
        plt.imsave("%s/%s%d.png" % (OUTPUT_FRAMES_PATH,frame_name, count), tmp, cmap= plt.cm.gray) # saves images to frame folder
        print ('*', end="")
    else:
        success,image = vidcap.read()                               # reads next frame
    count += 1                                                        # loops counter